In [1]:
import os
import sys
import xarray as xr
import numpy as np
from numpy import unravel_index
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.font_manager import FontProperties
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
%%time

# bins for BL_CAPE and BL_SUBSAT
bins_cape = np.arange(-15,10,0.5)
bins_subsat = np.arange(-5,25,0.5)
bins_samples = np.zeros((2, 6, len(bins_cape)-1, len(bins_subsat)-1)) # (area_type, mcs_phase, cape, subsat)
thetae_bl_sum = np.copy(bins_samples)

year_sel = 2001
env_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/envs_track/{}/tropics_extend'.format(year_sel))
files = list(env_dir.glob('*.LD.nc'))

for file in files[:100]:
    
    mcs_env = xr.open_dataset(file)

    # define stratiform as pixels with rainfall < 0.5mm/hr, Tb < 241 K
    
    for p,phase in enumerate(mcs_env.mcs_phase.values):
        
        data_phase = mcs_env.sel(mcs_phase=phase)
        thetae_bl = data_phase.thetae_bl
        thetae_lt = data_phase.thetae_lt
        thetae_sat_lt = data_phase.thetae_sat_lt
        sp = data_phase.SP/100 #hPa

        delta_pl=sp-100-400
        delta_pb=100
        wb=(delta_pb/delta_pl)*np.log((delta_pl+delta_pb)/delta_pb)
        wl=1-wb

        BL_CAPE = wb*(thetae_bl - thetae_sat_lt)/thetae_sat_lt*340 # (K)
        BL_SUBSAT = wl*(thetae_sat_lt - thetae_lt)/thetae_sat_lt*340 # (K)

        # define convective and stratiform pixels 
        tmp = data_phase.precipitationCal
        tmp2 = data_phase.tb
        tmp3 = data_phase.cloudtracknumber_nomergesplit
        
        conv_mcs_pix = tmp.where(((tmp >= 0.5) & (tmp2 < 241) & (tmp3 > 0)), 0)
        conv_mcs_pix = conv_mcs_pix.where(conv_mcs_pix ==0, 1)
        conv_mcs_pix = conv_mcs_pix.where(conv_mcs_pix ==1) # 1 or NaN
        
        strat_mcs_pix = tmp.where(((tmp < 0.5) & (tmp2 < 241) & (tmp3 > 0)), 0)
        strat_mcs_pix = strat_mcs_pix.where(strat_mcs_pix ==0, 1)
        strat_mcs_pix = strat_mcs_pix.where(strat_mcs_pix ==1) # 1 or NaN
        
        thetae_bl_1d = -thetae_bl.values.ravel()
        cape_1d = BL_CAPE.values.ravel()
        cape_1d = cape_1d[~np.isnan(conv_mcs_pix.values.ravel())]
        subsat_1d = BL_SUBSAT.values.ravel()
        subsat_1d = subsat_1d[~np.isnan(conv_mcs_pix.values.ravel())]
        thetae_bl_1d = thetae_bl_1d[~np.isnan(conv_mcs_pix.values.ravel())]
        
        for i in range(len(bins_cape)-1):
            idx = np.where(np.logical_and(cape_1d >= bins_cape[i], cape_1d < bins_cape[i+1]))[0]
            for j in range(len(bins_subsat)-1):
                idy = np.where(np.logical_and(subsat_1d >= bins_subsat[j], subsat_1d < bins_subsat[j+1]))[0]
                
                idx_com = np.intersect1d(idx,idy)
                if len(idx_com) > 0:
                    bins_samples[0,p,i,j] += len(idx_com)
                    thetae_bl_sum[0,p,i,j] += np.sum(thetae_bl_1d[idx_com])
                    
        thetae_bl_1d = -thetae_bl.values.ravel()                
        cape_1d = BL_CAPE.values.ravel()
        cape_1d = cape_1d[~np.isnan(strat_mcs_pix.values.ravel())]
        subsat_1d = BL_SUBSAT.values.ravel()
        subsat_1d = subsat_1d[~np.isnan(strat_mcs_pix.values.ravel())]
        thetae_bl_1d = thetae_bl_1d[~np.isnan(strat_mcs_pix.values.ravel())]
                                
        for i in range(len(bins_cape)-1):
            idx = np.where(np.logical_and(cape_1d >= bins_cape[i], cape_1d < bins_cape[i+1]))[0]
            for j in range(len(bins_subsat)-1):
                idy = np.where(np.logical_and(subsat_1d >= bins_subsat[j], subsat_1d < bins_subsat[j+1]))[0]
                
                idx_com = np.intersect1d(idx,idy)
                if len(idx_com) > 0:                    
                    bins_samples[1,p,i,j] += len(idx_com)
                    thetae_bl_sum[1,p,i,j] += np.sum(thetae_bl_1d[idx_com])
        
# writeout as dataset
ds = xr.Dataset(data_vars = dict(samples = (['area_type','phase','bins_cape','bins_subsat'], bins_samples),
                                 thetae_bl_sum = (['area_type','phase','bins_cape','bins_subsat'], thetae_bl_sum)),
                coords = dict(area_type = (['area_type'],['convective','stratiform']),
                              phase = (['phase'], ['CCS','Initial', 'Grow', 'Mature', 'Decay', 'End']),
                              bins_cape = (['bins_cape'], bins_cape[:-1]),
                              bins_subsat = (['bins_subsat'], bins_subsat[:-1])),
                attrs = dict(description = 'cape-subsat histogram. amean = 3-deg average'))

CPU times: user 1min 30s, sys: 7.98 s, total: 1min 38s
Wall time: 1min 25s


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))

conv_sample = ds.sel(area_type='convective', phase='Decay').samples
strat_sample = ds.sel(area_type='stratiform', phase='Decay').samples
conv_ratio = conv_sample / (conv_sample + strat_sample)
strat_ratio = strat_sample / (conv_sample + strat_sample)

# with sample size > 200
#conv_ratio = conv_ratio.where(conv_sample + strat_sample > 20)
#strat_ratio = strat_ratio.where(conv_sample + strat_sample > 20)

cp1=ax1.pcolormesh(ds.bins_subsat, ds.bins_cape, conv_ratio, vmin=0, vmax=1, cmap='RdBu')
plt.colorbar(cp1,ax=ax1)
ax1.set_title('Ratio of convective rainfall proxy',fontsize=9)
cp2=ax2.pcolormesh(ds.bins_subsat, ds.bins_cape, strat_ratio, vmin=0, vmax=1, cmap='RdBu')
plt.colorbar(cp2,ax=ax2)
ax2.set_title('Ratio of stratiform rainfall proxy',fontsize=9)

# add buoyancy reference 
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-8, color='grey', ls='--',lw=0.5)

# add buoyancy reference 
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-8, color='grey', ls='--',lw=0.5)

    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax2.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax1.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax2.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_ylim([-8,6])
ax1.set_xlim([-2,14])
ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.show()

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(3,3))

thetae_bl_phase = ds.sel(phase='Mature').sum('area_type').thetae_bl_sum
conv_sample = ds.sel(area_type='convective', phase='Mature').samples
strat_sample = ds.sel(area_type='stratiform', phase='Mature').samples
total_sample = conv_sample + strat_sample

thetae_bl_mean = thetae_bl_phase/total_sample

# with sample size > 200
#conv_ratio = conv_ratio.where(conv_sample + strat_sample > 20)
#strat_ratio = strat_ratio.where(conv_sample + strat_sample > 20)

cp1=ax1.pcolormesh(ds.bins_subsat, ds.bins_cape, thetae_bl_mean)
plt.colorbar(cp1,ax=ax1)
ax1.set_title('Conditional mean of thetae_bl',fontsize=9)

# add buoyancy reference 
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-8, color='grey', ls='--',lw=0.5)
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax1.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_ylim([-8,6])
ax1.set_xlim([-2,14])

plt.show()

In [ ]:
mcs_env = xr.open_dataset(files[0])
tmp = mcs_env.precipitationCal.sel(mcs_phase='Mature')
tmp2 = mcs_env.tb.sel(mcs_phase='Mature')
conv_pix = tmp.where(((tmp >= 0.5) & (tmp2 < 241)))
strat_pix = tmp.where(((tmp < 0.5) & (tmp2 < 241)))

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,3))

conv_pix.plot(ax=ax1, vmin=0,vmax=10)
mcs_env.cloudtracknumber_nomergesplit.sel(mcs_phase='Mature').plot.contour(ax=ax1)
strat_pix.plot(ax=ax2, vmin=0, vmax=10)
mcs_env.cloudtracknumber_nomergesplit.sel(mcs_phase='Mature').plot.contour(ax=ax2)
plt.show()